In [2]:
!pip install pandas xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00


In [18]:
import pandas as pd

# Sample data
file_path='/content/rawdata.xlsx'
data = pd.read_excel(file_path)
# Create DataFrame
df = pd.DataFrame(data)

# Ensure date and time are strings
df['date'] = df['date'].astype(str)
df['time'] = df['time'].astype(str)

# Convert date and time to datetime format
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])

# Normalize position values
df['position'] = df['position'].str.lower()

# Sort by datetime
df = df.sort_values(by='datetime')

# Derive total duration for each inside and outside position
df['next_datetime'] = df['datetime'].shift(-1)
df['duration'] = (df['next_datetime'] - df['datetime']).dt.total_seconds()

# Filter out the rows where next_datetime is NaT (last row)
df = df[df['next_datetime'].notna()]

# Group by date and position to calculate total duration
duration_df = df.groupby([df['datetime'].dt.date, 'position'])['duration'].sum().reset_index()
duration_df.columns = ['date', 'position', 'total_duration']

# Pivot the duration dataframe to get inside and outside durations in separate columns
duration_pivot_df = duration_df.pivot(index='date', columns='position', values='total_duration').reset_index()
duration_pivot_df = duration_pivot_df.rename_axis(None, axis=1).fillna(0)
duration_pivot_df.columns = ['date', 'inside_duration', 'outside_duration']

# Group by date and activity to count number of activities
activity_count_df = df.groupby([df['datetime'].dt.date, 'activity']).size().reset_index(name='count')
activity_count_df.columns = ['date', 'activity', 'count']

# Pivot the activity count dataframe to get pick and place activities in separate columns
activity_pivot_df = activity_count_df.pivot(index='date', columns='activity', values='count').reset_index()
activity_pivot_df = activity_pivot_df.rename_axis(None, axis=1).fillna(0)
activity_pivot_df.columns = ['date', 'pick_activities', 'place_activities']

# Merge the duration and activity dataframes on the date column
final_df = pd.merge(duration_pivot_df, activity_pivot_df, on='date', how='left').fillna(0)

# Print results
print("Final DataFrame:")
print(final_df)

# Save to Excel file with formatting
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer:
    final_df.to_excel(writer, sheet_name='Summary', index=False)

    # Get the xlsxwriter objects
    workbook = writer.book
    sheet = writer.sheets['Summary']

    # Set column widths for better visibility
    for column in final_df:
        max_len = final_df[column].astype(str).map(len).max()
        col_idx = final_df.columns.get_loc(column)
        sheet.set_column(col_idx, col_idx, max_len + 2)

print("\nFinal DataFrame saved to 'output.xlsx'.")


Final DataFrame:
         date  inside_duration  outside_duration  pick_activities  \
0  2024-01-16         122890.0               0.0               40   
1  2024-01-17          86400.0               0.0               10   
2  2024-01-18          30777.0            2813.0               36   

   place_activities  
0                40  
1                 9  
2                39  

Final DataFrame saved to 'output.xlsx'.
